# Randomized optimization

Plaigiarism note: I partially took this course in 2020 so some of the analysis and text is repeated.

mlrose procedure:

1. Define a fitness function
- This is the function we want to maximize or minimize, and is used to evaluate the fitness of a state vector.
2. Define an optimization problem object
3. Select and run a randomized optimization algorithm

mlrose fitness functions: https://mlrose.readthedocs.io/en/stable/source/fitness.html

## Load libraries

In [1]:
import six
import sys
sys.modules['sklearn.externals.six'] = six
import mlrose
import numpy as np
import pandas as pd
import time
from sklearn.preprocessing import normalize
from sklearn.metrics import accuracy_score

## Set directories

In [2]:
directory_hw1 = "/Users/mikepecorino/Documents/machine_learning/HW1/"
directory_hw2 = "/Users/mikepecorino/Documents/machine_learning/HW2/"
output_four_peaks = directory_hw2 + "four_peaks/"
output_knapsack = directory_hw2 + "knapsack/"
output_flip_flop = directory_hw2 + "flip_flop/"
output_one_max = directory_hw2 + "one_max/"

## Common inputs for each optimization problem

In [3]:
#Problem specific inputs
#- - - - - - - - - - - - - - - - - - - - - - - - - - - - - -
#Lengths of the state vector
lengths = [1000]
#Number of unique values for each x (0/1)
max_val = 2
#Keeping the fitness curves
curve = True
#Max attempts to find a better neighbor
max_attempts = [20]
#Max iterations
max_iters = range(100, 2100, 100)

#Algorithm specific inputs
#- - - - - - - - - - - - - - - - - - - - - - - - - - - - - -
#Random Hill Climbing and Simulated Annealing: setting initial state vectors to None
init_state = None
init_temps = [10, 50, 100, 150, 200, 250]
exp_const = 0.01
min_temp = 0.001

## Four Peaks

In [4]:
#Inputs for four peaks problem
#- - - - - - - - - - - - - - - - - - - - - - - - - - - - - -
#Initialize the fitness function with default t_pct
fitness_fn = mlrose.FourPeaks(t_pct = 0.1)

#Initialize an empty data frame for recording results
#- - - - - - - - - - - - - - - - - - - - - - - - - - - - - -
four_peaks_results = pd.DataFrame(columns = ["algorithm",
                                             "length",
                                             "init_temp",
                                             "max_attempt",
                                             "max_iter",
                                             "best_fitness",
                                             "time",
                                             "function_evaluations"])


#Loop
#- - - - - - - - - - - - - - - - - - - - - - - - - - - - - -
#Start the iteration counter
iter = 1
#For each unique number of objects
for length in lengths:
    
    #Set the optimization problem object
    problem = mlrose.DiscreteOpt(length = length,
                                 fitness_fn = fitness_fn,
                                 maximize = True,
                                 max_val = max_val)
    
    #For each combination of max attempt, max iteration, and random restart
    for max_attempt in max_attempts:
        for max_iter in max_iters:
            for init_temp in init_temps:
                
                #Ensuring reproducibility
                random_state = iter
                
                #Simulated Annealing: decay schedule for temperature
                schedule = mlrose.ExpDecay(init_temp = init_temp,
                                           exp_const = exp_const,
                                           min_temp = min_temp)

                #Start a timer for each iteration
                start = time.time()
                    
                #Print message
                print("Working on iter:", iter,
                      "Length:", length,
                      "Init_temp:", init_temp,
                      "Max attempt:", max_attempt,
                      "Max iter:", max_iter)

                #Simulated Annealing
                #- - - - - - - - - - - - - - - - - - - - - - - - - - - - - -
                #Start the timer for Simulated Annealing
                sa_start = time.time()
                
                #Optimization
                (sa_best_state,
                 sa_best_fitness,
                 sa_fitness_curve) = mlrose.simulated_annealing(problem = problem,
                                                                schedule = schedule,
                                                                max_attempts = max_attempt,
                                                                max_iters = max_iter,
                                                                init_state = init_state,
                                                                curve = curve,
                                                                random_state = random_state)
                #End the timer
                sa_end = time.time()
                #Get the total time
                sa_time = sa_end - sa_start
                
                #Message
                print("    SA best fitness:", sa_best_fitness)
                
                #Add to results list
                four_peaks_results = four_peaks_results.append({"algorithm": "simulated_annealing",
                                                                "length": length,
                                                                "init_temp": init_temp,
                                                                "max_attempt": max_attempt,
                                                                "max_iter": max_iter,
                                                                "best_fitness": sa_best_fitness,
                                                                "time": sa_time,
                                                                "function_evaluations": np.argmax(sa_fitness_curve) + 1},
                                                               ignore_index = True)

                
                #End the timer for all four algorithms
                end = time.time()
                #Total time for all four algorithms
                iter_time = end - start
                
                #Message
                print("Iter time:", iter_time)
                
                #Increment the iteration counter
                iter = iter + 1
                
                print("\n")
                
#Output results
four_peaks_results.to_csv(output_four_peaks + "four_peaks_sa.csv", index = False)
pd.DataFrame(sa_fitness_curve).to_csv(output_four_peaks + "four_peaks_sa_curve.csv", index = False)

#Done
print("Done.")

Working on iter: 1 Length: 1000 Init_temp: 10 Max attempt: 20 Max iter: 100
    SA best fitness: 2.0
Iter time: 0.0064737796783447266


Working on iter: 2 Length: 1000 Init_temp: 50 Max attempt: 20 Max iter: 100
    SA best fitness: 0.0
Iter time: 0.004608154296875


Working on iter: 3 Length: 1000 Init_temp: 100 Max attempt: 20 Max iter: 100
    SA best fitness: 0.0
Iter time: 0.005009174346923828


Working on iter: 4 Length: 1000 Init_temp: 150 Max attempt: 20 Max iter: 100
    SA best fitness: 4.0
Iter time: 0.0048329830169677734


Working on iter: 5 Length: 1000 Init_temp: 200 Max attempt: 20 Max iter: 100
    SA best fitness: 1.0
Iter time: 0.004513978958129883


Working on iter: 6 Length: 1000 Init_temp: 250 Max attempt: 20 Max iter: 100
    SA best fitness: 3.0
Iter time: 0.004839897155761719


Working on iter: 7 Length: 1000 Init_temp: 10 Max attempt: 20 Max iter: 200
    SA best fitness: 1.0
Iter time: 0.00652003288269043


Working on iter: 8 Length: 1000 Init_temp: 50 Max att

/Users/mikepecorino/opt/anaconda3/lib/python3.8/site-packages/mlrose/algorithms.py:311: RuntimeWarning: overflow encountered in exp
  prob = np.exp(delta_e/temp)


 6.0
Iter time: 0.03019404411315918


Working on iter: 68 Length: 1000 Init_temp: 50 Max attempt: 20 Max iter: 1200
    SA best fitness: 5.0
Iter time: 0.026774883270263672


Working on iter: 69 Length: 1000 Init_temp: 100 Max attempt: 20 Max iter: 1200
    SA best fitness: 15.0
Iter time: 0.030745744705200195


Working on iter: 70 Length: 1000 Init_temp: 150 Max attempt: 20 Max iter: 1200
    SA best fitness: 5.0
Iter time: 0.0288698673248291


Working on iter: 71 Length: 1000 Init_temp: 200 Max attempt: 20 Max iter: 1200
    SA best fitness: 5.0
Iter time: 0.030082225799560547


Working on iter: 72 Length: 1000 Init_temp: 250 Max attempt: 20 Max iter: 1200
    SA best fitness: 1.0
Iter time: 0.028618812561035156


Working on iter: 73 Length: 1000 Init_temp: 10 Max attempt: 20 Max iter: 1300
    SA best fitness: 2.0
Iter time: 0.02931380271911621


Working on iter: 74 Length: 1000 Init_temp: 50 Max attempt: 20 Max iter: 1300
    SA best fitness: 2.0
Iter time: 0.02878713607788086


Wo

## Knapsack

In [5]:
#Inputs for knapsack problem
#- - - - - - - - - - - - - - - - - - - - - - - - - - - - - -
weights = np.random.randint(1, 6, lengths)
values = np.random.randint(1, 6, lengths)
max_weight_pct = 0.6
fitness_fn = mlrose.Knapsack(weights = weights, values = values, max_weight_pct = max_weight_pct)

#Initialize an empty data frame for recording results
#- - - - - - - - - - - - - - - - - - - - - - - - - - - - - -
knapsack_results = pd.DataFrame(columns = ["algorithm",
                                           "length",
                                           "init_temp",
                                           "max_attempt",
                                           "max_iter",
                                           "best_fitness",
                                           "time",
                                           "function_evaluations"])

#Loop
#- - - - - - - - - - - - - - - - - - - - - - - - - - - - - -
#Start an iteration counter
iter = 1
#For each unique number of objects
for length in lengths:
    
    #Set the optimization problem object
    problem = mlrose.DiscreteOpt(length = length,
                                 fitness_fn = fitness_fn,
                                 maximize = True,
                                 max_val = max_val)
    
    #For each combination of max attempt, max iteration, and random restart
    for max_attempt in max_attempts:
        for max_iter in max_iters:
            for init_temp in init_temps:
                
                #Ensuring reproducibility
                random_state = iter
                
                #Simulated Annealing: decay schedule for temperature
                schedule = mlrose.ExpDecay(init_temp = init_temp,
                                           exp_const = exp_const,
                                           min_temp = min_temp)
                
                #Start a timer for each iteration
                start = time.time()
                    
                #Print message
                print("Working on iter:", iter,
                      "Length:", length,
                      "Init_temp:", init_temp,
                      "Max attempt:", max_attempt,
                      "Max iter:", max_iter)

                #Simulated Annealing
                #- - - - - - - - - - - - - - - - - - - - - - - - - - - - - -
                #Start the timer for Simulated Annealing
                sa_start = time.time()
                
                #Optimization
                (sa_best_state,
                 sa_best_fitness,
                 sa_fitness_curve) = mlrose.simulated_annealing(problem = problem,
                                                                schedule = schedule,
                                                                max_attempts = max_attempt,
                                                                max_iters = max_iter,
                                                                init_state = init_state,
                                                                curve = curve,
                                                                random_state = random_state)
                #End the timer
                sa_end = time.time()
                #Get the total time
                sa_time = sa_end - sa_start
                
                #Message
                print("    SA best fitness:", sa_best_fitness)
                
                #Add to results list
                knapsack_results = knapsack_results.append({"algorithm": "simulated_annealing",
                                                            "length": length,
                                                            "init_temp": init_temp,
                                                            "max_attempt": max_attempt,
                                                            "max_iter": max_iter,
                                                            "best_fitness": sa_best_fitness,
                                                            "time": sa_time,
                                                            "function_evaluations": np.argmax(sa_fitness_curve) + 1},
                                                            ignore_index = True)
                
                #End the timer for all four algorithms
                end = time.time()
                #Total time for all four algorithms
                iter_time = end - start
                
                #Message
                print("Iter time:", iter_time)
                
                print("\n")
                iter = iter + 1

#Output results                
knapsack_results.to_csv(output_knapsack + "knapsack_sa.csv", index = False)
pd.DataFrame(sa_fitness_curve).to_csv(output_knapsack + "knapsack_sa_curve.csv", index = False)

#Done
print("Done.")

Working on iter: 1 Length: 1000 Init_temp: 10 Max attempt: 20 Max iter: 100
    SA best fitness: 1487.0
Iter time: 0.010740041732788086


Working on iter: 2 Length: 1000 Init_temp: 50 Max attempt: 20 Max iter: 100
    SA best fitness: 1540.0
Iter time: 0.011059999465942383


Working on iter: 3 Length: 1000 Init_temp: 100 Max attempt: 20 Max iter: 100
    SA best fitness: 1484.0
Iter time: 0.01147913932800293


Working on iter: 4 Length: 1000 Init_temp: 150 Max attempt: 20 Max iter: 100
    SA best fitness: 1519.0
Iter time: 0.010241985321044922


Working on iter: 5 Length: 1000 Init_temp: 200 Max attempt: 20 Max iter: 100
    SA best fitness: 1497.0
Iter time: 0.010213136672973633


Working on iter: 6 Length: 1000 Init_temp: 250 Max attempt: 20 Max iter: 100
    SA best fitness: 1488.0
Iter time: 0.010159969329833984


Working on iter: 7 Length: 1000 Init_temp: 10 Max attempt: 20 Max iter: 200
    SA best fitness: 1633.0
Iter time: 0.01357412338256836


Working on iter: 8 Length: 1000 

    SA best fitness: 1839.0
Iter time: 0.02898693084716797


Working on iter: 64 Length: 1000 Init_temp: 150 Max attempt: 20 Max iter: 1100
    SA best fitness: 1816.0
Iter time: 0.03298211097717285


Working on iter: 65 Length: 1000 Init_temp: 200 Max attempt: 20 Max iter: 1100
    SA best fitness: 1836.0
Iter time: 0.03130197525024414


Working on iter: 66 Length: 1000 Init_temp: 250 Max attempt: 20 Max iter: 1100
    SA best fitness: 1827.0
Iter time: 0.033720970153808594


Working on iter: 67 Length: 1000 Init_temp: 10 Max attempt: 20 Max iter: 1200
    SA best fitness: 1827.0
Iter time: 0.018728971481323242


Working on iter: 68 Length: 1000 Init_temp: 50 Max attempt: 20 Max iter: 1200
    SA best fitness: 1825.0
Iter time: 0.023829936981201172


Working on iter: 69 Length: 1000 Init_temp: 100 Max attempt: 20 Max iter: 1200
    SA best fitness: 1818.0
Iter time: 0.028508901596069336


Working on iter: 70 Length: 1000 Init_temp: 150 Max attempt: 20 Max iter: 1200
    SA best fitnes

## FlipFlop

In [6]:
#Inputs for FlipFlop problem
#- - - - - - - - - - - - - - - - - - - - - - - - - - - - - -
fitness_fn = mlrose.FlipFlop()

#Initialize an empty data frame for recording results
#- - - - - - - - - - - - - - - - - - - - - - - - - - - - - -
flip_flop_results = pd.DataFrame(columns = ["algorithm",
                                            "length",
                                            "init_temp",
                                            "max_attempt",
                                            "max_iter",
                                            "best_fitness",
                                            "time",
                                            "function_evaluations"])

#Loop
#- - - - - - - - - - - - - - - - - - - - - - - - - - - - - -
#Start an iteration counter
iter = 1
#For each unique number of objects
for length in lengths:
    
    #Set the optimization problem object
    problem = mlrose.DiscreteOpt(length = length,
                                 fitness_fn = fitness_fn,
                                 maximize = True,
                                 max_val = max_val)
    
    #For each combination of max attempt, max iteration, and random restart
    for max_attempt in max_attempts:
        for max_iter in max_iters:
            for init_temp in init_temps:
                
                #Ensuring reproducibility
                random_state = iter
                
                #Simulated Annealing: decay schedule for temperature
                schedule = mlrose.ExpDecay(init_temp = init_temp,
                                           exp_const = exp_const,
                                           min_temp = min_temp)
                    
                #Start a timer for each iteration
                start = time.time()
                    
                #Print message
                print("Working on iter:", iter,
                      "Length:", length,
                      "Init_temp:", init_temp,
                      "Max attempt:", max_attempt,
                      "Max iter:", max_iter)

                
                #Simulated Annealing
                #- - - - - - - - - - - - - - - - - - - - - - - - - - - - - -
                #Start the timer for Simulated Annealing
                sa_start = time.time()
                
                #Optimization
                (sa_best_state,
                 sa_best_fitness,
                 sa_fitness_curve) = mlrose.simulated_annealing(problem = problem,
                                                                schedule = schedule,
                                                                max_attempts = max_attempt,
                                                                max_iters = max_iter,
                                                                init_state = init_state,
                                                                curve = curve,
                                                                random_state = random_state)
                #End the timer
                sa_end = time.time()
                #Get the total time
                sa_time = sa_end - sa_start
                
                #Message
                print("    SA best fitness:", sa_best_fitness)
                
                #Add to results list
                flip_flop_results = flip_flop_results.append({"algorithm": "simulated_annealing",
                                                              "length": length,
                                                              "init_temp": init_temp,
                                                              "max_attempt": max_attempt,
                                                              "max_iter": max_iter,
                                                              "best_fitness": sa_best_fitness,
                                                              "time": sa_time,
                                                              "function_evaluations": np.argmax(sa_fitness_curve) + 1},
                                                               ignore_index = True)

                
                end = time.time()
                iter_time = end - start
                print("Iter time:", iter_time)
                
                print("\n")
                iter = iter + 1

#Output results
flip_flop_results.to_csv(output_flip_flop + "flip_flop_sa.csv", index = False)
pd.DataFrame(sa_fitness_curve).to_csv(output_flip_flop + "flip_flop_sa_curve.csv", index = False)

#Done
print("Done.")

Working on iter: 1 Length: 1000 Init_temp: 10 Max attempt: 20 Max iter: 100
    SA best fitness: 487.0
Iter time: 0.05822420120239258


Working on iter: 2 Length: 1000 Init_temp: 50 Max attempt: 20 Max iter: 100
    SA best fitness: 471.0
Iter time: 0.05867505073547363


Working on iter: 3 Length: 1000 Init_temp: 100 Max attempt: 20 Max iter: 100
    SA best fitness: 495.0
Iter time: 0.058275699615478516


Working on iter: 4 Length: 1000 Init_temp: 150 Max attempt: 20 Max iter: 100
    SA best fitness: 500.0
Iter time: 0.06171107292175293


Working on iter: 5 Length: 1000 Init_temp: 200 Max attempt: 20 Max iter: 100
    SA best fitness: 510.0
Iter time: 0.0606389045715332


Working on iter: 6 Length: 1000 Init_temp: 250 Max attempt: 20 Max iter: 100
    SA best fitness: 494.0
Iter time: 0.061972856521606445


Working on iter: 7 Length: 1000 Init_temp: 10 Max attempt: 20 Max iter: 200
    SA best fitness: 549.0
Iter time: 0.11116528511047363


Working on iter: 8 Length: 1000 Init_temp: 

    SA best fitness: 738.0
Iter time: 0.5186140537261963


Working on iter: 62 Length: 1000 Init_temp: 50 Max attempt: 20 Max iter: 1100
    SA best fitness: 709.0
Iter time: 0.5448601245880127


Working on iter: 63 Length: 1000 Init_temp: 100 Max attempt: 20 Max iter: 1100
    SA best fitness: 718.0
Iter time: 0.5506739616394043


Working on iter: 64 Length: 1000 Init_temp: 150 Max attempt: 20 Max iter: 1100
    SA best fitness: 688.0
Iter time: 0.5613069534301758


Working on iter: 65 Length: 1000 Init_temp: 200 Max attempt: 20 Max iter: 1100
    SA best fitness: 700.0
Iter time: 0.5686049461364746


Working on iter: 66 Length: 1000 Init_temp: 250 Max attempt: 20 Max iter: 1100
    SA best fitness: 692.0
Iter time: 0.5743908882141113


Working on iter: 67 Length: 1000 Init_temp: 10 Max attempt: 20 Max iter: 1200
    SA best fitness: 753.0
Iter time: 0.5638000965118408


Working on iter: 68 Length: 1000 Init_temp: 50 Max attempt: 20 Max iter: 1200
    SA best fitness: 717.0
Iter time:

### OneMax

In [7]:
#Inputs for FlipFlop problem
#- - - - - - - - - - - - - - - - - - - - - - - - - - - - - -
fitness_fn = mlrose.OneMax()

#Initialize an empty data frame for recording results
#- - - - - - - - - - - - - - - - - - - - - - - - - - - - - -
one_max_results = pd.DataFrame(columns = ["algorithm",
                                          "length",
                                          "init_temp",
                                          "max_attempt",
                                          "max_iter",
                                          "best_fitness",
                                          "time",
                                          "function_evaluations"])

#Loop
#- - - - - - - - - - - - - - - - - - - - - - - - - - - - - -
#Start an iteration counter
iter = 1
#For each unique number of objects
for length in lengths:
    
    #Set the optimization problem object
    problem = mlrose.DiscreteOpt(length = length,
                                 fitness_fn = fitness_fn,
                                 maximize = True,
                                 max_val = max_val)
    
    #For each combination of max attempt, max iteration, and random restart
    for max_attempt in max_attempts:
        for max_iter in max_iters:
            for init_temp in init_temps:
                
                #Ensuring reproducibility
                random_state = iter
                
                #Simulated Annealing: decay schedule for temperature
                schedule = mlrose.ExpDecay(init_temp = init_temp,
                                           exp_const = exp_const,
                                           min_temp = min_temp)
                
                #Start a timer for each iteration
                start = time.time()
                    
                #Print message
                print("Working on iter:", iter,
                      "Length:", length,
                      "Init_temp:", init_temp,
                      "Max attempt:", max_attempt,
                      "Max iter:", max_iter)

                #Simulated Annealing
                #- - - - - - - - - - - - - - - - - - - - - - - - - - - - - -
                #Start the timer for Simulated Annealing
                sa_start = time.time()
                
                #Optimization
                (sa_best_state,
                 sa_best_fitness,
                 sa_fitness_curve) = mlrose.simulated_annealing(problem = problem,
                                                                schedule = schedule,
                                                                max_attempts = max_attempt,
                                                                max_iters = max_iter,
                                                                init_state = init_state,
                                                                curve = curve,
                                                                random_state = random_state)
                #End the timer
                sa_end = time.time()
                #Get the total time
                sa_time = sa_end - sa_start
                
                #Message
                print("    SA best fitness:", sa_best_fitness)
                
                #Add to results list
                one_max_results = one_max_results.append({"algorithm": "simulated_annealing",
                                                          "length": length,
                                                          "init_temp": init_temp,
                                                          "max_attempt": max_attempt,
                                                          "max_iter": max_iter,
                                                          "best_fitness": sa_best_fitness,
                                                          "time": sa_time,
                                                          "function_evaluations": np.argmax(sa_fitness_curve) + 1},
                                                          ignore_index = True)
                
                end = time.time()
                iter_time = end - start
                print("Iter time:", iter_time)
                
                print("\n")
                iter = iter + 1

#Output results
one_max_results.to_csv(output_one_max + "one_max_sa.csv", index = False)
pd.DataFrame(sa_fitness_curve).to_csv(output_one_max + "one_max_sa_curve.csv", index = False)

#Done
print("Done.")

Working on iter: 1 Length: 1000 Init_temp: 10 Max attempt: 20 Max iter: 100
    SA best fitness: 483.0
Iter time: 0.010927200317382812


Working on iter: 2 Length: 1000 Init_temp: 50 Max attempt: 20 Max iter: 100
    SA best fitness: 506.0
Iter time: 0.007892847061157227


Working on iter: 3 Length: 1000 Init_temp: 100 Max attempt: 20 Max iter: 100
    SA best fitness: 490.0
Iter time: 0.007279872894287109


Working on iter: 4 Length: 1000 Init_temp: 150 Max attempt: 20 Max iter: 100
    SA best fitness: 505.0
Iter time: 0.011870861053466797


Working on iter: 5 Length: 1000 Init_temp: 200 Max attempt: 20 Max iter: 100
    SA best fitness: 501.0
Iter time: 0.011682748794555664


Working on iter: 6 Length: 1000 Init_temp: 250 Max attempt: 20 Max iter: 100
    SA best fitness: 502.0
Iter time: 0.04352903366088867


Working on iter: 7 Length: 1000 Init_temp: 10 Max attempt: 20 Max iter: 200
    SA best fitness: 526.0
Iter time: 0.014731884002685547


Working on iter: 8 Length: 1000 Init_t

    SA best fitness: 687.0
Iter time: 0.06740999221801758


Working on iter: 61 Length: 1000 Init_temp: 10 Max attempt: 20 Max iter: 1100
    SA best fitness: 781.0
Iter time: 0.05983901023864746


Working on iter: 62 Length: 1000 Init_temp: 50 Max attempt: 20 Max iter: 1100
    SA best fitness: 771.0
Iter time: 0.03391218185424805


Working on iter: 63 Length: 1000 Init_temp: 100 Max attempt: 20 Max iter: 1100
    SA best fitness: 753.0
Iter time: 0.033354997634887695


Working on iter: 64 Length: 1000 Init_temp: 150 Max attempt: 20 Max iter: 1100
    SA best fitness: 718.0
Iter time: 0.03550004959106445


Working on iter: 65 Length: 1000 Init_temp: 200 Max attempt: 20 Max iter: 1100
    SA best fitness: 727.0
Iter time: 0.06934809684753418


Working on iter: 66 Length: 1000 Init_temp: 250 Max attempt: 20 Max iter: 1100
    SA best fitness: 713.0
Iter time: 0.0866539478302002


Working on iter: 67 Length: 1000 Init_temp: 10 Max attempt: 20 Max iter: 1200
    SA best fitness: 666.0
Ite

    SA best fitness: 821.0
Iter time: 0.05185198783874512


Working on iter: 120 Length: 1000 Init_temp: 250 Max attempt: 20 Max iter: 2000
    SA best fitness: 712.0
Iter time: 0.03690171241760254


Done.
